In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from models_and_migration import session, RealEstateListing
from sqlalchemy import create_engine
from sqlalchemy.inspection import inspect
import warnings


Data Collection and Processing

In [65]:
# loading the data from csv file to pandas dataframe
engine = create_engine("sqlite:///mydb.db", echo=True)
connection = engine.connect()
columns = ['url', 'city', 'district', 'year', 'area', 'room_number', 'floor_number', 'no_of_floors', 'building_type', 'heating_system', 'indoor_installation', 'price', 'latest_followed_by_no_people', 'viewed_by_no_people_overall']
print(columns)
df_raw = pd.DataFrame(columns=columns)
rows_realestatelistings = session.query(RealEstateListing).filter(RealEstateListing.city.in_(('Vilnius', 'Kaunas', 'Klaipėda'))).all()
for row in rows_realestatelistings:
    warnings.filterwarnings(action='ignore', category=FutureWarning)
    df_raw = df_raw.append({'url': row.url,
                    'city': row.city,
                    'district': row.district,
                    'year': row.year,
                    'area': row.area,
                    'room_number': row.room_number,
                    'floor_number': row.floor_number,
                    'no_of_floors': row.no_of_floors,
                    'building_type': row.building_type,
                    'heating_system': row.heating_system,
                    'indoor_installation': row.indoor_installation,
                    'price': row.price,
                    'latest_followed_by_no_people': row.latest_followed_by_no_people,
                    'viewed_by_no_people_overall': row.viewed_by_no_people_overall,
                    }, ignore_index=True)
# df.to_csv(f'mlaruodas.csv', index=False, encoding='utf-8')

['url', 'city', 'district', 'year', 'area', 'room_number', 'floor_number', 'no_of_floors', 'building_type', 'heating_system', 'indoor_installation', 'price', 'latest_followed_by_no_people', 'viewed_by_no_people_overall']
2022-12-06 18:43:41,296 INFO sqlalchemy.engine.Engine SELECT real_estate_listings.url AS real_estate_listings_url, real_estate_listings.status AS real_estate_listings_status, real_estate_listings.apartment_or_house AS real_estate_listings_apartment_or_house, real_estate_listings.rent_or_sale AS real_estate_listings_rent_or_sale, real_estate_listings.city AS real_estate_listings_city, real_estate_listings.district AS real_estate_listings_district, real_estate_listings.street_name AS real_estate_listings_street_name, real_estate_listings.home_number AS real_estate_listings_home_number, real_estate_listings.apartment_number AS real_estate_listings_apartment_number, real_estate_listings.year AS real_estate_listings_year, real_estate_listings.area AS real_estate_listings_ar

In [66]:
df_raw.head()

,url,city,district,year,area,room_number,floor_number,no_of_floors,building_type,heating_system,indoor_installation,price,latest_followed_by_no_people,viewed_by_no_people_overall
0,www.aruodas.lt/1-3090567,Klaipėda,Šauliai,2022,45.0,2,2,4,Mūrinis,Centrinis kolektorinis,Dalinė apdaila,79990,113,27158
1,www.aruodas.lt/1-3268074,Klaipėda,Centras,1980,66.76,3,5,5,Mūrinis,Centrinis,Įrengtas,165000,15,586
2,www.aruodas.lt/1-3266846,Klaipėda,Senamiestis,2006,64.0,2,4,10,Mūrinis,Centrinis kolektorinis,Įrengtas,135000,15,497
3,www.aruodas.lt/1-3262872,Klaipėda,Žvejybos uostas,1974,78.46,4,5,5,Blokinis,Centrinis,Įrengtas,97000,12,1234
4,www.aruodas.lt/1-3267584,Klaipėda,Sportininkai,1970,36.0,1,3,3,Mūrinis,Kietu kuru,Įrengtas,46000,14,847


In [67]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4533 entries, 0 to 4532
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   url                           4533 non-null   object
 1   city                          4533 non-null   object
 2   district                      4533 non-null   object
 3   year                          4533 non-null   object
 4   area                          4533 non-null   object
 5   room_number                   4533 non-null   object
 6   floor_number                  4533 non-null   object
 7   no_of_floors                  4533 non-null   object
 8   building_type                 4533 non-null   object
 9   heating_system                4533 non-null   object
 10  indoor_installation           4533 non-null   object
 11  price                         4533 non-null   object
 12  latest_followed_by_no_people  4533 non-null   object
 13  viewed_by_no_peopl

In [68]:
df = df_raw[df_raw.price >= 5000]
df.dropna(inplace=True)

In [69]:
df.replace({'city':{'Vilnius':0, 'Kaunas':1, 'Klaipėda': 2}}, inplace=True)
df.replace({'building_type':{'Blokinis':0, 'Karkasinis':1, 'Kita': 2, 'Medinis': 3, 'Monolitinis': 4, 'Mūrinis': 5, "Rąstinis": 6}}, inplace=True)
df.replace({'indoor_installation':{'Dalinė apdaila':0, 'Įrengtas':1, 'Kita': 2, 'Nebaigtas statyti': 3, 'Neįrengtas': 4}}, inplace=True)
df.replace({'district':{'Aleksotas':1, 'Alksnynė':2, 'Antakalnis':3, 'Aukštieji Paneriai':4, 'Aukštieji Šančiai':5, 'Aukštutiniai Kaniūkai':6, 'Bajorai':7, 'Balsiai':8, 'Baltija':9, 'Baltupiai':10, 'Bandužiai':11, 'Buivydiškės':12, 'Bukčiai':13, 'Burbiškės':14, 'Centras':15, 'Dainava':16, 'Debrecenas':17, 'Didieji Gulbinai':18, 'Dvarčionys':19, 'Eiguliai':20, 'Fabijoniškės':21, 'Filaretai':22, 'Freda':23, 'Galgiai':24, 'Giruliai':25, 'Gričiupis':26, 'Grigiškės':27, 'Jeruzalė':28, 'Justiniškės':29, 'Kalnėnai':30, 'Kalniečiai':31, 'Karoliniškės':32, 'Kaunas':33, 'Kirtimai':34, 'Labrenciškės':35, 'Lampėdžiai':36, 'Laukininkai':37, 'Lazdynai':38, 'Lazdynėliai':39, 'Liepkalnis':40, 'Markučiai':41, 'Marvelė':42, 'Mažasis kaimelis':43, 'Melnragė':44, 'Miškas':45, 'Mokykla':46, 'Naujakiemis':47, 'Naujamiestis':48, 'Naujininkai':49, 'Naujoji Vilnia':50, 'Pagiriai':51, 'Palemonas':52, 'Panemunė':53, 'Paneriai':54, 'Pašilaičiai':55, 'Paupiai':56, 'Paupys':57, 'Pavilnys':58, 'Petrašiūnai':59, 'Pilaitė':60, 'Poilsis':61, 'Rasos':62, 'Rimkai':63, 'Rokai':64, 'Romainiai':65, 'Rumpiškės':66, 'Salininkai':67, 'Santariškės':68, 'Sargėnai':69, 'Šauliai':70, 'Senamiestis':71, 'Šeškinė':72, 'Šiaurės miestelis':73, 'Šilainiai':74, 'Smeltė':75, 'Šnipiškės':76, 'Sportininkai':77, 'Tarandė':78, 'Tauralaukis':79, 'Trakų Vokė':80, 'Trekas':81, 'Užupis':82, 'Vaišvydava':83, 'Valakampiai':84, 'Varpai':85, 'Verkiai':86, 'Vėtrungė':87, 'Vičiūnai':88, 'Vilijampolė':89, 'Vilkpėdė':90, 'Vingis':91, 'Viršuliškės':92, 'Visoriai':93, 'Vytėnai':94, 'Žaliakalnis':95, 'Žardininkai':96, 'Žemieji Paneriai':97, 'Žemieji Šančiai':98, 'Žemutiniai Kaniūkai':99, 'Žirmūnai':100, 'Žvejybos uostas':101, 'Žvėrynas':102}}, inplace=True)
df['floor_type'] = np.select([(df['floor_number'] == 0), (df['floor_number'] == 1), (df['floor_number'] == df['no_of_floors'])], [0, 1, 2], default=3)
df['heating_electricity']= df['heating_system'].apply(lambda x: 1 if ('elektra' in x.lower()) else 0)
df['heating_central']= df['heating_system'].apply(lambda x: 1 if ((('centrinis' in x.lower()) and ('kolektorinis' not in x.lower())) or ('centrinis kolektorinis, centrinis' in x.lower())) else 0)
df['heating_central_collective ']= df['heating_system'].apply(lambda x: 1 if ('centrinis kolektorinis' in x.lower()) else 0)
df['heating_liquid_fuel']= df['heating_system'].apply(lambda x: 1 if ('skystu kuru' in x.lower()) else 0)
df['heating_gas']= df['heating_system'].apply(lambda x: 1 if ('dujinis' in x.lower()) else 0)
df['heating_sun']= df['heating_system'].apply(lambda x: 1 if ('saulės energija' in x.lower()) else 0)
df['heating_aerothermic']= df['heating_system'].apply(lambda x: 1 if ('aeroterminis' in x.lower()) else 0)
df['heating_solid_fuel']= df['heating_system'].apply(lambda x: 1 if ('kietu kuru' in x.lower()) else 0)
df['heating_geothermal']= df['heating_system'].apply(lambda x: 1 if ('geoterminis' in x.lower()) else 0)
df['heating_other']= df['heating_system'].apply(lambda x: 1 if ('kita' in x.lower()) else 0)

In [70]:
df = df.astype({'year': 'int64', 'area': 'float64', 'room_number': 'int64', 'floor_number': 'int64', 'no_of_floors': 'int64', 'price': 'int64', 'latest_followed_by_no_people': 'int64', 'viewed_by_no_people_overall': 'int64'})

In [71]:
df_append = df.copy(deep = True)

In [72]:
df.head()

,url,city,district,year,area,room_number,floor_number,no_of_floors,building_type,heating_system,...,heating_electricity,heating_central,heating_central_collective,heating_liquid_fuel,heating_gas,heating_sun,heating_aerothermic,heating_solid_fuel,heating_geothermal,heating_other
0,www.aruodas.lt/1-3090567,2,70,2022,45.00,2,2,4,5,Centrinis kolektorinis,...,0,0,1,0,0,0,0,0,0,0
1,www.aruodas.lt/1-3268074,2,15,1980,66.76,3,5,5,5,Centrinis,...,0,1,0,0,0,0,0,0,0,0
2,www.aruodas.lt/1-3266846,2,71,2006,64.00,2,4,10,5,Centrinis kolektorinis,...,0,0,1,0,0,0,0,0,0,0
3,www.aruodas.lt/1-3262872,2,101,1974,78.46,4,5,5,0,Centrinis,...,0,1,0,0,0,0,0,0,0,0
4,www.aruodas.lt/1-3267584,2,77,1970,36.00,1,3,3,5,Kietu kuru,...,0,0,0,0,0,0,0,1,0,0


In [73]:
df.dtypes

url                              object
city                              int64
district                          int64
year                              int64
area                            float64
room_number                       int64
floor_number                      int64
no_of_floors                      int64
building_type                     int64
heating_system                   object
indoor_installation               int64
price                             int64
latest_followed_by_no_people      int64
viewed_by_no_people_overall       int64
floor_type                        int32
heating_electricity               int64
heating_central                   int64
heating_central_collective        int64
heating_liquid_fuel               int64
heating_gas                       int64
heating_sun                       int64
heating_aerothermic               int64
heating_solid_fuel                int64
heating_geothermal                int64
heating_other                     int64


Full Sample target variable and independent variables

In [74]:
X = df.drop(['url', 'price', 'heating_system'],axis=1)
Y = df['price']

Splitting to training and testing data

In [75]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.30, random_state=4)

Linear Regresion

In [76]:
lm = LinearRegression()
lm.fit(X_train, Y_train)
Y_pred = lm.predict(X_train)
print('R^2 for training data:', metrics.r2_score(Y_train, Y_pred))

R^2 for training data: 0.6135741176262518


In [77]:
# Predinting Test data with the model
Y_test_pred = lm.predict(X_test)
print('R^2 for test data:', metrics.r2_score(Y_test, Y_test_pred))

R^2 for test data: 0.598143631032835


Lasso Regression

In [78]:
lass_reg_model = Lasso()
lass_reg_model.fit(X_train,Y_train)
training_data_prediction = lass_reg_model.predict(X_train)
print("R squared Error : ", metrics.r2_score(Y_train, training_data_prediction))

R squared Error :  0.6135739957407014


In [79]:
Y_test_pred = lass_reg_model.predict(X_test)
print('R^2 for test data:', metrics.r2_score(Y_test, Y_test_pred))

R^2 for test data: 0.5981403436699003


Random Forest

In [80]:
reg = RandomForestRegressor()
reg.fit(X_train, Y_train)
Y_pred = reg.predict(X_train)
print('R^2 for training data:', metrics.r2_score(Y_train, Y_pred))


R^2 for training data: 0.9642695273852219


In [81]:
Y_test_pred = reg.predict(X_test)
print('R^2 for test data:', metrics.r2_score(Y_test, Y_test_pred))

R^2 for test data: 0.7910911730931153


XGBoost

In [82]:
reg = XGBRegressor()
reg.fit(X_train, Y_train)
Y_pred = reg.predict(X_train)
print('R^2 for training data:', metrics.r2_score(Y_train, Y_pred))

R^2 for training data: 0.9933659710025241


In [83]:
Y_test_pred = reg.predict(X_test)
print('R^2 for testing data:', metrics.r2_score(Y_test, Y_test_pred))

R^2 for testing data: 0.8328397245523506


In [89]:
pred_results = reg.predict(X)
price_prediction = pd.Series(data=pred_results, name='pred_results')
print(price_prediction)


0        77697.804688
1       105240.914062
2       175865.593750
3        93650.023438
4        41896.417969
            ...      
4528     82170.937500
4529     30455.082031
4530    355005.062500
4531     96772.250000
4532    179126.312500
Name: pred_results, Length: 4533, dtype: float32


In [92]:
df_out = pd.concat([df_append.reset_index(), price_prediction], axis=1)
df_out.head()

,index,url,city,district,year,area,room_number,floor_number,no_of_floors,building_type,...,heating_central,heating_central_collective,heating_liquid_fuel,heating_gas,heating_sun,heating_aerothermic,heating_solid_fuel,heating_geothermal,heating_other,pred_results
0,0,www.aruodas.lt/1-3090567,2,70,2022,45.00,2,2,4,5,...,0,1,0,0,0,0,0,0,0,77697.804688
1,1,www.aruodas.lt/1-3268074,2,15,1980,66.76,3,5,5,5,...,1,0,0,0,0,0,0,0,0,105240.914062
2,2,www.aruodas.lt/1-3266846,2,71,2006,64.00,2,4,10,5,...,0,1,0,0,0,0,0,0,0,175865.593750
3,3,www.aruodas.lt/1-3262872,2,101,1974,78.46,4,5,5,0,...,1,0,0,0,0,0,0,0,0,93650.023438
4,4,www.aruodas.lt/1-3267584,2,77,1970,36.00,1,3,3,5,...,0,0,0,0,0,0,1,0,0,41896.417969


In [94]:
df_out = df_out.astype({'pred_results': 'int64'})

In [95]:
df_out.to_csv('ml_aruodas.csv', index=False, encoding='utf-8')
